![Reference Image for Beating Angle](papon.png)

In [1]:

formula = r"\widehat{P_1P_0P_2} = \arccos\left(\frac{\overline{P_0P_1}^2 + \overline{P_0P_2}^2 - \overline{P_1P_2}^2}{2\overline{P_0P_1}\overline{P_0P_2}}\right)"
print(formula)


\widehat{P_1P_0P_2} = \arccos\left(\frac{\overline{P_0P_1}^2 + \overline{P_0P_2}^2 - \overline{P_1P_2}^2}{2\overline{P_0P_1}\overline{P_0P_2}}\right)


In [ ]:
dataset_path = "/Users/sukhvir

In [3]:
import numpy as np 


def euclidean_distance(p1, p2):
    """
    Calculate Euclidean Distance Between two points
    
    Args:
        p1, p2: (x,y) coordinates 
    
    Returns:
        distance (float)
    """
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)


def compute_beat_angle(p0, p1, p2):
    """
    Compute the angle at point p0 formed by points p0, p1, and p2
    
    Args:
        p0, p1, p2: (x,y) coordinates 
    
    Returns:
        angle (float): angle in degrees
    """
    a = euclidean_distance(p1, p2)
    b = euclidean_distance(p0, p2)
    c = euclidean_distance(p0, p1)

    # Compute cosine of the angle using the law of cosines
    cos_angle = (b**2 + c**2 - a**2) / (2 * b * c)

    # Clamp the value to avoid numerical errors
    cos_angle = np.clip(cos_angle, -1.0, 1.0)

    angle_rad = np.arccos(cos_angle)
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg


def compute_beat_angle_from_trajectory(p0, tip_trajectory):
    """
    Compute the beat angle from a trajectory of tip positions
    
    Args:
        p0: Basal point (fixed)
        tip_trajectory: Array of shape (num_frames, 2) with (x, y) coordinates
    
    Returns:
        beat_angles: Array of angles
        max_angle: Maximum angle in the trajectory
    """
    beat_angles = []
    
    for i in range(len(tip_trajectory) - 1):
        p1 = tip_trajectory[i]
        
        # Find opposite extreme (opposite phase)
        opposite_idx = (i + len(tip_trajectory) // 2) % len(tip_trajectory)
        p2 = tip_trajectory[opposite_idx]
        
        angle = compute_beat_angle(p0, p1, p2)
        beat_angles.append(angle)
    
    return np.array(beat_angles), np.max(beat_angles)

if __name__ == "__main__":
    # Healthy cilium
    P0 = np.array([100, 100])
    P1 = np.array([150, 80])
    P2 = np.array([120, 140])
    
    angle = compute_beat_angle(P0, P1, P2)
    print(f"Beat Angle (healthy): {angle:.2f}°")
    
    # Dyskinetic cilium
    P1_dys = np.array([115, 95])
    P2_dys = np.array([105, 105])
    
    angle_dys = compute_beat_angle(P0, P1_dys, P2_dys)
    print(f"Beat Angle (dyskinetic): {angle_dys:.2f}°")


Beat Angle (healthy): 85.24°
Beat Angle (dyskinetic): 63.43°


**Processing own data-base based for beading angle calculations and importing into a csv file**

In [5]:
#Import libraries
import numpy as np
import cv2 
import os 
from pathlib import Path 
import scipy.signal as find_peaks
import matplotlib.pyplot as plt

In [ ]:

def load_video_dataset(dataset_path):
    """
    Load all HSVM videos from dataset folder
    
    Args:
        dataset_path: Path to folder containing .mp4, .avi, .mov files
    
    Returns:
        video_list: List of video file paths
        labels: List of labels (e.g., 'healthy', 'pcd')
    """
    video_extensions = ['.mp4', '.avi', '.mov', '.MOV', '.MP4']
    video_list = []
    labels = []

    dataset_path = Path(dataset_path)
    #Scan the directory 
    for video in sorted(dataset_path.iterdir()):
        if video.is_file() and video.suffix.lower() in video_extensions:
            video_list.append(video)

            # Try to infer label from filename
            if 'healthy' in video.name.lower():
                labels.append('healthy')
            elif 'pcd' in video.name.lower():
                labels.append('pcd')
            else:
                labels.append('unknown')
        print(f"loaded {len(video_list)} videos")

    def load_frames(video_path, max_frames=None):
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            print(f"Error opening video {video_path}")
            return None, None, None 
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    frame_count = 0 

    while True:
        ret, frame = cap.read()

        if not ret:
            break
        frames.append(frame)
        frame_count += 1
        if max_frames and frame_count >= max_frames:  
            break

    cap.release()  
    return frames, fps, total_frames


    print(f"Loaded {len(frames)} frames at {fps} fps")
    return frames, fps, len(frames)

## BEAT ANGLE (P₁P₀P₂) - Mathematical Foundation & Biological Significance

### Mathematical Formula (Law of Cosines)

$$ \widehat{P_1P_0P_2} = \arccos\left(\frac{\overline{P_0P_1}^2 + \overline{P_0P_2}^2 $$ 


In [ ]:
def euclidean_distance(p1, p2):
    """Calculate the Euclidean distance between two points."""
    return np.sqrt(np.sum((p1 - p2) ** 2) + (p1[1] - p2[1]) ** 2)


def compute_beat_angle(p1, p2, p3):
    """Compute the angle formed by three vectors using Arcos and position vectors."""
    """
    Compute beat angle using law of cosine

    Args: p0, p1, p2 : (x,y) coordinates

    returns: angle (float): in degrees
    """ 

    a = euclidean_distance(p1, p2)
    b = euclidean_distance(p2, p3)
    c = euclidean_distance(p1, p3)

    # Law of cosines
    angle = np.arccos((a**2 + b**2 - c**2) / (2 * a * b))
    return np.degrees(angle)
    angle_rad = np.arccos(cos_angle)
    angle_deg = np.degrees(angle_rad)


#detect the tip of the cilia 
def detect_cilium_tip(frame, roi_box = None):
    """
    Detect the tip of the cilium in a frame.

    Args: 
    frame: single frame (BGR)
    roi_box: (x1, y1, x2, y2)

    Returns: 
        tip_position = (x,y) coordinates 
    """

    if roi box is not None:
        x1, y1, x2, y2 = roi_box
    roi = frame[y1:y2, x1:x2]

    # Convert to grayscale
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    # Detect edges
    edges = cv2.Canny(gray, 50, 150)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    
    # Get largest contour (cilium)
    cnt = max(contours, key=cv2.contourArea)
    
    # Find centroid
    M = cv2.moments(cnt)
    if M["m00"] > 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        
        if roi_box is not None:
            cx += roi_box[0]
            cy += roi_box[1]
        
        return np.array([cx, cy])
    
    return None

    def track_cillium_tips(frames, roi_box=None):
        """ 
        Track the tip of the cilium across a sequence of frames.
        """
        tip_positions = []
        for frame in frames:
            tip_pos = detect_cilium_tip(frame, roi_box)
            tip_positions.append(tip_pos)
        return tip_positions
    tip_positions = []
    for i, frame in enumerate(frames):
        tip_pos = detect_cilium_tip(frame, roi_box)
        tip_positions.append(tip_pos)
    return tip_positions

    def compute_beat_angle_from_trajectory(p0, tip_trajectory):
        """Compute beat angle from trajectory.
        """
        beat_angles = []
        for i in range(len(tip_trajectory)):
            p1 = tip_trajectory[i]

            #opposite extreme
        opposite_idx = (i + len(tip_trajectory) // 2) % len(tip_trajectory)
        p2 = tip_trajectory[opposite_idx]
        
        angle = compute_beat_angle(p0, p1, p2)
        beat_angles.append(angle)
    
    beat_angles = np.array(beat_angles)
    max_angle = np.max(beat_angles)
    mean_angle = np.mean(beat_angles)
    std_angle = np.std(beat_angles)
    
    return beat_angles, max_angle, mean_angle, std_angle